In [2]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
load_dotenv()

llm = llm = ChatAnthropic(model_name='claude-opus-4-0')

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [5]:
def generate_jokes(state: JokeState):

    prompt = f'generate a joke on the topic  {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [6]:

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_jokes', generate_jokes)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_jokes')
graph.add_edge('generate_jokes','generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic': 'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!',
 'explanation': 'This joke is a pun that plays on the double meaning of "toppings" and "deal with."\n\nIn the literal sense, a pizza maker works with pizza toppings - ingredients like pepperoni, mushrooms, cheese, etc. Having "too many toppings" would be a normal work situation for someone who makes pizzas.\n\nBut the phrase "too many things to deal with" is a common expression meaning someone is overwhelmed by problems or stress in their life. The joke cleverly substitutes "toppings" for "things," creating a wordplay between the pizza-making context and the mental health context.\n\nThe humor comes from this unexpected connection between pizza ingredients and emotional baggage - suggesting that the pizza maker needs therapy not for personal issues, but literally because of pizza toppings. It\'s a lighthearted play on words that combines the professional and psychological mea

In [9]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!', 'explanation': 'This joke is a pun that plays on the double meaning of "toppings" and "deal with."\n\nIn the literal sense, a pizza maker works with pizza toppings - ingredients like pepperoni, mushrooms, cheese, etc. Having "too many toppings" would be a normal work situation for someone who makes pizzas.\n\nBut the phrase "too many things to deal with" is a common expression meaning someone is overwhelmed by problems or stress in their life. The joke cleverly substitutes "toppings" for "things," creating a wordplay between the pizza-making context and the mental health context.\n\nThe humor comes from this unexpected connection between pizza ingredients and emotional baggage - suggesting that the pizza maker needs therapy not for personal issues, but literally because of pizza toppings. It\'s a lighthearted play on words that combines the professional an

In [10]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!', 'explanation': 'This joke is a pun that plays on the double meaning of "toppings" and "deal with."\n\nIn the literal sense, a pizza maker works with pizza toppings - ingredients like pepperoni, mushrooms, cheese, etc. Having "too many toppings" would be a normal work situation for someone who makes pizzas.\n\nBut the phrase "too many things to deal with" is a common expression meaning someone is overwhelmed by problems or stress in their life. The joke cleverly substitutes "toppings" for "things," creating a wordplay between the pizza-making context and the mental health context.\n\nThe humor comes from this unexpected connection between pizza ingredients and emotional baggage - suggesting that the pizza maker needs therapy not for personal issues, but literally because of pizza toppings. It\'s a lighthearted play on words that combines the professional a

Time Travel

In [15]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06eaf0-622c-69c9-8000-ebe2d22872f4"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_jokes',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06eaf0-622c-69c9-8000-ebe2d22872f4'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T08:10:44.976040+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06eaf0-614f-6678-bfff-c7866cd33160'}}, tasks=(PregelTask(id='a2ba3cb1-5b0e-aca8-dfe3-1d40430ed878', name='generate_jokes', path=('__pregel_pull', 'generate_jokes'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!'}),), interrupts=())

In [16]:
workflow.invoke(None , {"configurable": {"thread_id": "1", "checkpoint_id": "1f06eaf0-622c-69c9-8000-ebe2d22872f4"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!',
 'explanation': 'This joke is a pun that plays on the double meaning of "toppings" and "deal with."\n\nIn the literal sense, a pizza maker works with pizza toppings - ingredients like pepperoni, mushrooms, cheese, etc. Having "too many toppings" would just be a normal work situation for someone who makes pizzas.\n\nBut the punchline creates humor by connecting this to therapy. In psychology and everyday language, when someone has "too much to deal with," it means they\'re overwhelmed by problems, stress, or emotional issues - which is why people typically seek therapy.\n\nThe joke works by substituting "toppings" for "things" in the common phrase "too many things to deal with," creating a food pun that links the pizza maker\'s job to mental health struggles. It\'s a lighthearted play on words that imagines pizza toppings as emotional baggage or psychological burdens that need

In [17]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!', 'explanation': 'This joke is a pun that plays on the double meaning of "toppings" and "deal with."\n\nIn the literal sense, a pizza maker works with pizza toppings - ingredients like pepperoni, mushrooms, cheese, etc. Having "too many toppings" would just be a normal work situation for someone who makes pizzas.\n\nBut the punchline creates humor by connecting this to therapy. In psychology and everyday language, when someone has "too much to deal with," it means they\'re overwhelmed by problems, stress, or emotional issues - which is why people typically seek therapy.\n\nThe joke works by substituting "toppings" for "things" in the common phrase "too many things to deal with," creating a food pun that links the pizza maker\'s job to mental health struggles. It\'s a lighthearted play on words that imagines pizza toppings as emotional baggage or psychologic

Updating Stae

In [20]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06eaf0-622c-69c9-8000-ebe2d22872f4", "checkpoint_ns": "" }}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f06eb07-42ac-6572-8001-c4e7a27a78bd'}}

In [21]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_jokes',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06eb07-42ac-6572-8001-c4e7a27a78bd'}}, metadata={'source': 'update', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T08:20:59.074464+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06eaf0-622c-69c9-8000-ebe2d22872f4'}}, tasks=(PregelTask(id='c062defe-b211-bc7b-b573-26a7b3c1da6a', name='generate_jokes', path=('__pregel_pull', 'generate_jokes'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker go to therapy?\n\nHe had too many toppings to deal with!', 'explanation': 'This joke is a pun that plays on the double meaning of "toppings" and "deal with."\n\nIn the literal sense, a pizza maker works with pizza toppings - ingredients like pepperoni, mushrooms, cheese, etc. Having "

In [22]:
workflow.invoke(None , {"configurable": {"thread_id": "1", "checkpoint_id": "1f06eb07-42ac-6572-8001-c4e7a27a78bd"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\n\nBecause it had too many *layers* to work through and kept feeling *fried* under pressure!\n\n🥟😄',
 'explanation': 'This joke is a delightful play on words that uses the physical characteristics of a samosa to create a mental health metaphor!\n\n**The Setup:** The joke personifies a samosa (a popular triangular fried pastry from South Asian cuisine) as someone seeking psychological help.\n\n**The Punchline Works on Two Levels:**\n\n1. **"Too many layers"** - Samosas literally have multiple layers of thin, flaky pastry that are folded to create their distinctive triangular shape. But in therapy-speak, having "layers" refers to complex emotional issues, past traumas, or psychological defenses that need to be "worked through" or unpacked.\n\n2. **"Feeling fried"** - Samosas are deep-fried as part of their cooking process, making them crispy and golden. But "feeling fried" is also a common expression meaning feeling exhauste